In [6]:
!pip install pandas opencv-python

                                              0.0/38.1 MB ? eta -:--:--
                                              0.1/38.1 MB 2.6 MB/s eta 0:00:15
                                              0.2/38.1 MB 2.3 MB/s eta 0:00:17
                                              0.3/38.1 MB 2.2 MB/s eta 0:00:18
                                              0.4/38.1 MB 2.4 MB/s eta 0:00:16
                                              0.5/38.1 MB 2.3 MB/s eta 0:00:17
                                              0.7/38.1 MB 2.4 MB/s eta 0:00:16
                                              0.8/38.1 MB 2.3 MB/s eta 0:00:16
                                              0.9/38.1 MB 2.4 MB/s eta 0:00:16
     -                                        1.0/38.1 MB 2.4 MB/s eta 0:00:16
     -                                        1.1/38.1 MB 2.4 MB/s eta 0:00:16
     -                                        1.2/38.1 MB 2.3 MB/s eta 0:00:16
     -                                        1.3/38.1 MB 2


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
N = ord('R') % 5 + 1

print(N)

3


In [50]:
import pandas as pd
import cv2

class ImageProcessor:
    def __init__(self, variant_data):
        # Конструктор класу ImageProcessor ініціалізує об'єкт з вхідними даними варіанту
        self.variant_data = variant_data
        self.colors = {
            'red': (0, 0, 255),
            'blue': (255, 0, 0),
            'cian': (255, 255, 0),
            'magenta': (255, 0, 255),
            'yellow': (0, 255, 255)
        }

    def convert_color(self, color_text):
        # Метод для конвертації текстового опису кольору в BGR формат
        return self.colors.get(color_text, (0, 0, 0))

    def process_image(self, image_path):
        # Метод обробки зображення: зчитування, додавання окулярів та збереження обробленого зображення
        img = cv2.imread(image_path)
        if img is None:
            print(f"Не вдалося зчитати зображення {image_path}")
            return None

        # Ініціалізація Haar Cascade класифікаторів для визначення облич та очей
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

        # Перетворення зображення в градації сірого кольору та визначення облич
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Отримання даних з вхідного датасету для обробки зображення
        glasses_color_bgr = self.convert_color(self.variant_data['glasses color'].values[0])
        image_size = tuple(map(int, self.variant_data['image size'].values[0].split('x')))
        line_width = int(self.variant_data['line width'].values[0])

        for (x, y, w, h) in faces:
            # Для кожного обличчя на зображенні
            face_roi = img[y:y+h, x:x+w]
            face_gray = gray[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(face_gray)
            eye_centers = []

            # Знаходження очей на обличчі
            for (ex, ey, ew, eh) in eyes:
                glass_x = int(ex + 0.5 * ew)
                glass_y = int(ey + 0.5 * eh)
                glass_radius = int(0.5 * max(ew, eh))

                # Додавання окулярів на обличчя
                cv2.circle(face_roi, (glass_x, glass_y), glass_radius, glasses_color_bgr, line_width)
                eye_centers.append((x + glass_x, y + glass_y))

            eye_centers = sorted(eye_centers, key=lambda x: x[0])

            if len(eye_centers) == 2:
                # Додавання "дужок" та "перенісся" до окулярів
                cv2.line(img, (eye_centers[0][0] + glass_radius, eye_centers[0][1]), (eye_centers[1][0] - glass_radius, eye_centers[1][1]), glasses_color_bgr, line_width)
                cv2.line(img, (eye_centers[0][0] - glass_radius, eye_centers[0][1]), (eye_centers[0][0] - 50, eye_centers[0][1]), glasses_color_bgr, line_width)
                cv2.line(img, (eye_centers[1][0] + glass_radius, eye_centers[1][1]), (eye_centers[1][0] + 50, eye_centers[1][1]), glasses_color_bgr, line_width)

            # Зміна розміру зображення та збереження результуючого зображення
            result_img = cv2.resize(img, image_size)
            result_image_path = f'{image_path[:-4]}_glasses.jpg'
            cv2.imwrite(result_image_path, result_img)

            return result_image_path

# Читання файлу з даними
df = pd.read_excel('lab6.xlsx')

# Вибір данних для конкретного варіанту
variant_data = df[df['N'] == 3]

# Шлях до зображення для обробки
file_path = "./Images/" + variant_data['file name'].values[0]

# Створення об'єкту ImageProcessor та обробка зображення
processor = ImageProcessor(variant_data)
result_image_path = processor.process_image(file_path)

# Виведення шляху до результату
print(f"Зображення збережено у файлі: {result_image_path}")

Зображення збережено у файлі: ./Images/draco_glasses.jpg
